Dataset from https://www.kaggle.com/datasets/groffo/ads16-dataset

[1] Roffo, G., & Vinciarelli, A. (2016, August). Personality in computational advertising: A benchmark. In 4 th Workshop on Emotions and Personality in Personalized Systems (EMPIRE) 2016 (p. 18).

In [1]:
from keras.layers import Input, Conv3D, MaxPooling3D, Dense, Lambda, Flatten, Concatenate, LSTM, SimpleRNN
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from pandas import read_csv
import tensorflow as tf
import numpy as np
import pytesseract
import datetime
import string
import cv2
import sys
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

if __name__ == "__main__":
    mpl.rcParams["figure.figsize"] = (20, 6)

    %load_ext tensorboard


In [2]:
def get_ad_and_user_directories(root_directories):
    ad_directories = []
    user_directories = []

    for directory in root_directories:
        ad_directories_parent = os.path.join(directory, directory, "Ads", "Ads")
        user_directories_parent = os.path.join(directory, directory, "Corpus", "Corpus")

        ad_directories += list(os.path.join(ad_directories_parent, x) for x in os.listdir(ad_directories_parent))
        user_directories += list(os.path.join(user_directories_parent, x) for x in os.listdir(user_directories_parent))

    ad_directories = sorted(ad_directories, key=lambda x: int(x.replace("/", "\\").split("\\")[-1]))
    user_directories = sorted(user_directories, key=lambda x: int(x.replace("/", "\\").split("\\")[-1][1:]))

    num_categories = len(ad_directories)
    return ad_directories, user_directories, num_categories

if __name__ == "__main__":
    ad_directories, user_directories, num_categories = get_ad_and_user_directories(["ADS16_Benchmark_part1", "ADS16_Benchmark_part2"])

In [3]:
img_width = 16
img_height = 16
img_channels = 3

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        cap = cv2.VideoCapture(path)
        ret, img = cap.read()
        cap.release()
        assert ret
    if img.shape[2] > 3:
        y,x = img[:,:,3].nonzero()
        assert y.shape[0] > 0 and x.shape[0] > 0
        minx = np.min(x)
        miny = np.min(y)
        maxx = np.max(x)
        maxy = np.max(y) 
        img = img[miny:maxy, minx:maxx, :img_channels]
    img = cv2.resize(img, (img_width, img_height))
    assert img.shape == (img_width, img_height, img_channels)
    return img.astype(float) / 255.

In [4]:
def load_ads(ad_directories, num_categories):
    ad_categories = np.zeros((0, num_categories))
    ad_imgs = np.zeros((0, img_width, img_height, img_channels))
    ad_text = []
    allowed_chars = list(string.ascii_lowercase + string.digits + " .")
    char_embedding_size = len(allowed_chars)+1

    last_ad_directory_id = ad_directories[-1].replace("/", "\\").split("\\")[-1]
    for ad_directory in ad_directories:
        ad_directory_id = ad_directory.replace("/", "\\").split("\\")[-1]
        img_paths = list(os.path.join(ad_directory, x) for x in os.listdir(ad_directory))
        categories = np.zeros((len(img_paths), num_categories))
        categories[:,int(ad_directory.replace("/", "\\").split("\\")[-1])-1] = 1
        img_arrays = np.array(list(map(load_image, img_paths)))
        ad_categories = np.concatenate((ad_categories, categories))
        ad_imgs = np.concatenate((img_arrays, ad_imgs))
        ad_text += list(list(allowed_chars.index(x) if x in allowed_chars else len(allowed_chars) for x in i.lower()) for i in map(pytesseract.image_to_string, img_paths))
        print(f"Loaded ad category {ad_directory_id}/{last_ad_directory_id}", end="\r")

    print("\nEmbedding text...")

    num_ads = ad_imgs.shape[0]
    assert len(ad_text) == num_ads
    max_text_length = max(map(len, ad_text))
    ad_char_embeddings = np.zeros((num_ads, max_text_length, char_embedding_size))
    for i, txt in enumerate(ad_text):
        for j, c in enumerate(txt):
            ad_char_embeddings[i, j, c] = 1
        
    print("Done")

    return ad_categories, ad_imgs, ad_char_embeddings, num_ads, char_embedding_size, max_text_length

if __name__ == "__main__":
    ad_categories, ad_imgs, ad_char_embeddings, num_ads, char_embedding_size, max_text_length = load_ads(ad_directories, num_categories)

Loaded ad category 20/20
Embedding text...
Done


In [5]:
def user_data_iterator(user_directories):
    failures = 0
    for user_directory in user_directories:
        user_id = user_directory.replace("/", "\\").split("\\")[-1]
        b5 = read_csv(os.path.join(user_directory, f"{user_id}-B5.csv"), delimiter=";")["Answer"].values
        im_neg = read_csv(os.path.join(user_directory, f"{user_id}-IM-NEG.csv"), delimiter=";")
        im_neg_paths = list(os.path.join(user_directory, f"{user_id}-IM-NEG", *x.replace("/", "\\").split("\\")[1:]) for x in im_neg.iloc[0].values)[:5]


        im_pos = read_csv(os.path.join(user_directory, f"{user_id}-IM-POS.csv"), delimiter=";")
        im_pos_paths = list(os.path.join(user_directory, f"{user_id}-IM-POS", *x.replace("/", "\\").split("\\")[1:]) for x in im_pos.iloc[0].values)[:5]
        if len(im_neg_paths) != 5 or len(im_pos_paths) != 5:
            failures += 1
            print(f"Failed to load user {user_id} (not enough images): {failures} total failure{'' if failures == 1 else 's'}")
            continue

        try:
            im_neg_arrays = np.array(list(map(load_image, im_neg_paths)))
            im_pos_arrays = np.array(list(map(load_image, im_pos_paths)))
        except AssertionError:
            failures += 1
            print(f"Failed to load user {user_id} (bad images): {failures} total failure{'' if failures == 1 else 's'}")
            continue

        # im_neg_labels = im_neg.iloc[1].values
        # im_pos_labels = im_pos.iloc[1].values

        inf = read_csv(os.path.join(user_directory, f"{user_id}-INF.csv"), delimiter=";")
        gender_age_income = np.array([[1 if x[0] == "F" else 0 if x[0] == "M" else 0.5, *x[1:]] for x in inf[["Gender", "Age", "Income"]].values][0])
        
        rt = read_csv(os.path.join(user_directory, f"{user_id}-RT.csv"), delimiter=";")
        mean_rt_per_cat = np.mean(np.array(list(list(int(i) for i in x.split(",")) for x in rt.iloc[1].values)), axis=1)
        ratings = np.array(list(map(int, ",".join(rt.iloc[1].values).split(","))))
        ratings_one_hot = np.eye(5)[ratings-1]

        yield (user_id, b5, im_neg_arrays, im_pos_arrays, gender_age_income, mean_rt_per_cat, ratings_one_hot)


In [7]:
if __name__ == "__main__":
    X0 = np.zeros((0, 10+3+num_categories+num_categories)) # B5, Gender, Age, Income, Mean Rating per Category, Ad Category
    X1 = np.zeros((0, 5+5+1, img_width, img_height, img_channels)) # 5 Positive Images, 5 Negative Images, Ad Image
    X2 = np.zeros((0, max_text_length, char_embedding_size)) # Ad text
    Y = np.zeros((0, 5)) # Rating

    last_user_id = user_directories[-1].replace("/", "\\").split("\\")[-1]
    for (
        user_id,
        b5,
        im_neg_arrays,
        im_pos_arrays,
        gender_age_income,
        mean_rt_per_cat,
        ratings_one_hot
    ) in user_data_iterator(user_directories):
        x0 = np.broadcast_to(b5, (num_ads, *b5.shape))
        x0 = np.concatenate((x0, np.broadcast_to(gender_age_income, (num_ads, *gender_age_income.shape))), axis=1)
        x0 = np.concatenate((x0, np.broadcast_to(mean_rt_per_cat, (num_ads, *mean_rt_per_cat.shape))), axis=1)
        x0 = np.concatenate((x0, ad_categories), axis=1)

        x1 = np.broadcast_to(im_pos_arrays, (num_ads, *im_pos_arrays.shape))
        x1 = np.concatenate((x1, np.broadcast_to(im_neg_arrays, (num_ads, *im_neg_arrays.shape))), axis=1)
        x1 = np.concatenate((x1, ad_imgs[:, np.newaxis, :, :, :]), axis=1)

        x2 = ad_char_embeddings

        y = ratings_one_hot

        X0 = np.concatenate((X0, x0))
        X1 = np.concatenate((X1, x1))
        X2 = np.concatenate((X2, x2))
        Y = np.concatenate((Y, y))

        print(f"Loaded user {user_id}/{last_user_id}", end="\r")


Failed to load user U0011 (not enough images): 1 total failure
Failed to load user U0024 (bad images): 2 total failures
Failed to load user U0039 (bad images): 3 total failures
Failed to load user U0042 (bad images): 4 total failures


In [8]:
if __name__ == "__main__":
    np.savez_compressed("data/user-data.npz", X0=X0, X1=X1, X2=X2, Y=Y)

In [9]:
d = np.load("data/user-data.npz")
X0 = d["X0"]
X1 = d["X1"]
X2 = d["X2"]
Y = d["Y"]

In [10]:
def one_hots_to_ratings(one_hots):
    return tf.reduce_sum(one_hots * tf.range(1,6,dtype=tf.dtypes.float32), axis=-1)

def ratings_mae(y_true, y_pred):
    ratings_true = one_hots_to_ratings(y_true)
    ratings_pred = one_hots_to_ratings(y_pred)
    return tf.reduce_mean(tf.math.abs(ratings_true-ratings_pred))

def create_model():
    input0 = Input(X0.shape[1:])
    input1 = Input(X1.shape[1:])
    input2 = Input(X2.shape[1:])

    output1 = Conv3D(4, (1,3,3), activation="relu", kernel_regularizer="l2")(input1)
    output1 = MaxPooling3D(pool_size=(1,2,2))(output1)
    output1 = Conv3D(2, (1,3,3), activation="relu", kernel_regularizer="l2")(output1)
    output1 = MaxPooling3D(pool_size=(1,4,4))(output1)
    output1 = Flatten()(output1)
    output2 = SimpleRNN(4)(input2)
    output2 = Flatten()(output2)
    output = Concatenate()([input0, output1, output2])
    output = Dense(2, activation="relu", kernel_regularizer="l2")(output)
    output = Dense(5, activation="softmax", kernel_regularizer="l2")(output)

    return Model(inputs=[input0, input1, input2], outputs=[output])

if __name__ == "__main__":
    model = create_model()
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy", ratings_mae])
    model.summary()
    plot_model(model, to_file="users_model.png", show_shapes=True, show_layer_activations=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 11, 16, 16,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 11, 14, 14,   112         ['input_2[0][0]']                
                                4)                                                                
                                                                                                  
 max_pooling3d (MaxPooling3D)   (None, 11, 7, 7, 4)  0           ['conv3d[0][0]']                 
                                                                                              

In [11]:
if __name__ == "__main__":
    total_num = Y.shape[0]
    batch_size = 64

    train_prop = 0.8
    train_num = int(train_prop * total_num)
    train_split_num = np.ceil(train_num/batch_size)

    # permutation = np.random.permutation(total_num)
    permutation = np.arange(total_num)
    X = (X0[permutation], X1[permutation], X2[permutation])
    Y = Y[permutation]

    X_train = list(np.array_split(x[:train_num], train_split_num) for x in X)
    X_train = list(map(list, zip(*X_train)))

    X_val = list(x[train_num:] for x in X)

    Y_train = np.array_split(Y[:train_num], train_split_num)
    Y_val = Y[train_num:]

In [12]:
if __name__ == "__main__":

    checkpoints_dir="users-checkpoints"
    filepath = os.path.join(checkpoints_dir, "model-{epoch:06d}-{val_ratings_mae:.4f}.hdf5")
    checkpoint = ModelCheckpoint(filepath, save_freq=len(X_train), save_weights_only=True, monitor="val_ratings_mae", verbose=0, save_best_only=False, mode="min")

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
if __name__ == "__main__":

    epochs = 100

    if not os.path.isdir(checkpoints_dir):
        os.makedirs(checkpoints_dir)

    try:
        for epoch in range(epochs):
            for i, split in enumerate(np.random.permutation(len(X_train))):
                model.fit(
                    x=X_train[split],
                    y=Y_train[split],
                    batch_size=len(X_train[split][0]),
                    initial_epoch=epoch,
                    epochs=epoch+1,
                    verbose=0,
                    shuffle=True,
                    callbacks=[checkpoint]
                )
                print(f"[{i+1}/{len(X_train)}]", end="                        \r")

            print(f"Epoch {epoch+1}, Step {(epoch+1)*len(X_train)}: ", end="")
            model.evaluate(
                x=X_val,
                y=Y_val,
                batch_size=batch_size,
                verbose=2,
                callbacks=[tensorboard_callback]
            )

    except KeyboardInterrupt:
        pass
    finally:
        plt.close()

Epoch 1, Step 435:               109/109 - 5s - loss: 1.5960 - categorical_accuracy: 0.6022 - ratings_mae: 1.0732 - 5s/epoch - 50ms/step
Epoch 2, Step 870:               109/109 - 6s - loss: 1.4204 - categorical_accuracy: 0.6022 - ratings_mae: 0.9984 - 6s/epoch - 53ms/step
Epoch 3, Step 1305:              109/109 - 5s - loss: 1.2912 - categorical_accuracy: 0.6022 - ratings_mae: 0.9660 - 5s/epoch - 47ms/step
Epoch 4, Step 1740:              109/109 - 6s - loss: 1.2562 - categorical_accuracy: 0.6022 - ratings_mae: 0.9142 - 6s/epoch - 52ms/step
Epoch 5, Step 2175:              109/109 - 5s - loss: 1.2278 - categorical_accuracy: 0.6022 - ratings_mae: 0.9873 - 5s/epoch - 46ms/step
Epoch 6, Step 2610:              109/109 - 6s - loss: 1.1950 - categorical_accuracy: 0.6022 - ratings_mae: 0.8758 - 6s/epoch - 54ms/step
Epoch 7, Step 3045:              109/109 - 5s - loss: 1.1774 - categorical_accuracy: 0.6022 - ratings_mae: 0.8988 - 5s/epoch - 48ms/step
Epoch 8, Step 3480:              109/109 